<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/RAG%EA%B8%B0%EB%B0%98FAQ%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community -q
!pip install langchain_openai -q

In [ ]:
!pip install chromadb

In [ ]:
!pip install pymupdf

In [ ]:
# 필수 라이브러리
from langchain.document_loaders import TextLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
os.environ['OPENAI_API_KEY'] = '...'

In [ ]:
# 문서로드 및 분할
def load_and_split_text(file_path):
  if file_path.endswith('.pdf'):
    loader = PyMuPDFLoader(file_path)
  else:
    loader = TextLoader(file_path)

  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20,
                    separators=['.','\n',' ',''],keep_separator=True       )
  return text_splitter.split_documents(documents)

In [ ]:
# RAG 챗봇구현
def create_rag_chatbot(texts):
  # 벡터 데이터베이스 생성
  vectorstore = Chroma.from_documents(texts, OpenAIEmbeddings(), persist_directory='./db3')
  # LLm 준비
  llm = OpenAI(temperature=0.2,max_tokens=2048 )
  # 프롬프트 엔지니어링(한글 답변 최적화)
  prompt_template = PromptTemplate(
      input_variables=['context', 'question'],
      template='''문서 : {context}
      질문:{question}
      다음 단계을 따라 한글로 답변하세요
      1. 문서에서 질문과 관련된 문장 검색.
      2. 문장이 있으면 문장기반으로 답변구성
      3. 문장이 없으면 "문서에 정보 없음"을 명시하고, 일반지식으로 간결히답변
      답변은 정확해야 합니다.
      '''

  )
  # RetrivalQA 체인
  rag_chatbot = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type='stuff',
      retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
      chain_type_kwargs={'prompt': prompt_template},
      return_source_documents = True,
  )
  return rag_chatbot


In [ ]:
# 문서로드 및 분할
texts = load_and_split_text("/content/산림병해충 방제규정(산림청훈령)(제1664호)(20241210).pdf")
print("문서개수:",len(texts) )
qa_chain = create_rag_chatbot(texts)

문서개수: 588


In [ ]:
query = '위험평가의 실시'
result = qa_chain.invoke(query)
print(f'\n\n질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')



질문 : 위험평가의 실시
답변 : 
위험평가위원회를 둔 후, 발생하거나 예상되는 외래ㆍ돌발병해충에 대해 긴급하게 위험평가를 실시합니다.
참조 문서 : ['(이하 "위험평가위원회"라 한다)를 둔다', '하고 그 결과를 위험평가에 반영할 수 있다', '가 발생하였거나 피해가 예상되는 외래ㆍ돌발병해충에 대해 긴급하게 위험평가를 실시한다']


벡터DB 데이터 검색

In [ ]:
# 벡터 데이터베이스 검색
documents = '''제4조의5(위험평가의 실시) ① 위험평가는 다음 각 호의 항목을 포함하여 실시하여야 한다.
1. 대상 산림병해충의 외래병해충 여부
2. 대상 산림병해충의 생리ㆍ생태적 특성
3. 대상 산림병해충으로 인한 예상 피해 정도
4. 긴급방제 추진의 필요성과 방제방법
② 제1항에 의한 산림병해충의 위험등급은 별표 1의 산림병해충 위험평가표를 기준으로 계량화된 점수를 산정
하고 별표 2의 산림병해충 종합위험도 판정기준에 따라 다음 각 호와 같이 판정한다.
1. 종합평가점수 ‘높음’ : ‘고위험 병해충’
2. 종합평가점수 ‘중간’ : ‘중위험 병해충’
3. 종합평가점수 ‘낮음’ : ‘저위험 병해충’
4. 산림과 생활환경 및 경제에 미치는 영향에 대한 위험요소 항목 모두가 ‘가장 높은 점수’로 평가되는 경우 다른
평가 항목의 평가점수가 낮아도 ‘고위험 병해충’으로 판정할 수 있다.
5. 산림과 생활환경 및 경제에 미치는 영향에 대한 위험요소 항목 모두가 ‘가장 낮은 점수’로 평가되는 경우 다른
평가 항목의 평가점수가 높아도 ‘저위험 병해충’으로 판정할 수 있다.
6. 병해충의 정보가 부족하여 평가가 제한적인 경우 종합위험도 판정을 유예하되, 산림과 생활환경 및 경제에 미
치는 영향에 대한 위험성이 예상된다면 위험관리방안 수준을 고려하여 종합위험도를 판정할 수 있다.
③ 기존에 평가된 병해충에 대하여 필요 시 재평가를 실시할 수 있다.
④ 위원장은 제4조에서 규정한 예찰조사 결과를 위험평가에 반영할 수 있다.
'''
with open('documents.txt','w') as f:
  f.write(documents)

def create_and_search_vectorstore(texts):
  embedding = OpenAIEmbeddings()
  vectorstore = Chroma.from_documents(texts, embedding, persist_directory='./db10')
  return vectorstore

texts = load_and_split_text("documents.txt")
vt = create_and_search_vectorstore(texts)

query = '위험평가의 실시'
vt.similarity_search(query,k=5)

[Document(metadata={'source': 'documents.txt'}, page_content='치는 영향에 대한 위험성이 예상된다면 위험관리방안 수준을 고려하여 종합위험도를 판정할 수 있다'),
 Document(metadata={'source': 'documents.txt'}, page_content='치는 영향에 대한 위험성이 예상된다면 위험관리방안 수준을 고려하여 종합위험도를 판정할 수 있다'),
 Document(metadata={'source': 'documents.txt'}, page_content='제4조의5(위험평가의 실시) ① 위험평가는 다음 각 호의 항목을 포함하여 실시하여야 한다.\n1. 대상 산림병해충의 외래병해충 여부\n2. 대상 산림병해충의 생리ㆍ생태적 특성\n3'),
 Document(metadata={'source': 'documents.txt'}, page_content='제4조의5(위험평가의 실시) ① 위험평가는 다음 각 호의 항목을 포함하여 실시하여야 한다.\n1. 대상 산림병해충의 외래병해충 여부\n2. 대상 산림병해충의 생리ㆍ생태적 특성\n3'),
 Document(metadata={'source': 'documents.txt'}, page_content='.\n③ 기존에 평가된 병해충에 대하여 필요 시 재평가를 실시할 수 있다.\n④ 위원장은 제4조에서 규정한 예찰조사 결과를 위험평가에 반영할 수 있다.')]

PDF Loaders

In [ ]:
filepath = '/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf'
result = load_and_split_text(filepath)
for doc in result:
  print(doc.page_content)


혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2호 | 1 |
<요  약>
<요  약>
▶혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신
성장 기술분야에 대한 금융지원을 강화하고 있음
* 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 
11개 기관
11개 기관
    ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성
￮
￮
정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24
.1조 원에서 2021년 85
.4조 원으로 크게 증가
하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음
￮
￮
본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의 
기술·시장 동향을 분석함
기술·시장 동향을 분석함
▶혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야,
19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16
.9조 원으로 
2017년 이후 연평균 39
.2% 지속 증가하고 있음
￮
ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 규모의 약 20% 수준임
      * (‘17)18.7% → (’18)20
.7% → (’18)20.7%  → (’19)18.5%  → (’20)20.3%  → (’21)19
.3%  → (’21)19.8%
혁신성장 정책금융 동향 :
ICT 산업을 중심으로
오동찬 선임조사역 (dcoh@kcredit.or.kr)
| 2 | CIS이슈리포트 2022-2호 
▶혁신성장 ICT 산업의 정책금융 공급규모 및 공급속도를 종합적으로 분석한 결과, 차세대무선통신미디어,
능동형컴퓨팅(이상 정보통신 테마), 차세대반도체(전기전자 테마) 및 객체탐지(센서측정 테마) 기술분야로

OCR 기능을 이용해서 이미지내 텍스트 추출

In [ ]:
!pip install rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 45.1 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 7.7 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(filepath,extract_images=True)
pages =  loader.load_and_split()  # 페이지별

In [ ]:
pages[0]

Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2022-07-29T09:03:16+09:00', 'author': 'kmd kdy', 'moddate': '2022-07-29T09:03:16+09:00', 'pdfversion': '1.4', 'source': '/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로\n  CIS이슈리포트 2022-2호 | 1 |\n<요  약>▶혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신성장 기술분야에 대한 금융지원을 강화하고 있음     * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 11개 기관    ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성￮정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음￮본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의 기술·시장 동향을 분석함▶혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 2017년 이후 연평균 39.2% 지속 증가하고 있음￮ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 규모의 약 20% 수준임      * (‘

In [ ]:
!pip install pypdfium2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.2 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFium2Loader
loader = PyPDFium2Loader(filepath,extract_images=True)
pages =  loader.load_and_split()  # 페이지별

/usr/local/lib/python3.11/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [ ]:
pages[0]

Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2022-07-29T09:03:16+09:00', 'title': '', 'author': 'kmd kdy', 'subject': '', 'keywords': '', 'moddate': '2022-07-29T09:03:16+09:00', 'source': '/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'total_pages': 18, 'page': 0}, page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로\n CIS이슈리포트 2022-2호 | 1 |\n<요 약>\n▶ 혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신\n성장 기술분야에 대한 금융지원을 강화하고 있음\n * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 \n11개 기관\n ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성\n￮ 정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가\n하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음\n￮ 본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의\n기술·시장 동향을 분석함\n▶ 혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 \n2017년 이후 연평균 39.2% 지속 증가하고 있음\n￮ ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 

In [ ]:
!pip install python-docx
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=07569c21b6dc23f7ace4d9dac3134b3fe3fe766212db70fe22b18643ca8e0b27
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
# 워드문서 로드
from langchain.document_loaders import UnstructuredWordDocumentLoader
loader = UnstructuredWordDocumentLoader("/content/문서1.docx")
pages = loader.load()  # 페이지 단위로 문서를 분리
pages

[Document(metadata={'source': '/content/문서1.docx'}, page_content='')]

이미지에 있는 text를 추출할때는
  - 1. 이미지를 word에 넣어서 pdf로 변환 - pdfloader
  - 2. 이미지 자체를 ocr 기능을 이용해서 텍스트를 추출

In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import easyocr
import cv2

In [ ]:
image = cv2.imread('/content/test.png')
reader = easyocr.Reader(['ko','en'])
results = reader.readtext(image)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

In [ ]:
for result in results:
  print(result[1])

3. 정보통신 테마 희신성장 정책금움 현황 및 관련 산업 동향
(지원 현황) 정보동신 테마을 구성하는 기술분야별 정책금웅 지원 현화 분석결과, 공급접유움 관점
예서는 차세대무선동신미디어 분야에 가장 많은 정책지금이 투입 되고 있으며 공급광 증가율 관점
예서는 능동형컴퓨팅 분야로의 정책지금 지원 증가 속도가 가장 빠른 추세임
차세대무선통신미디어란 전습숙도 향상 소모전력 절감, 고속이동 중 금김없는 통신 등 새로운 무선
환경에 피요한 동신  인프라 및 서비스 기숙울 동징하여   4G/5G/6G
사물인터넷 .방송동신인프라
등의 품목으로 구성님
정보통신
테마 내 허신성장 성책금움 공급 규모의 약 50*름 점유하고 있으머  이는 초연결 미래
사회들 구축하기 위해 네트위크 기반 기순 사업화에 대한 정책지금 공급이 꾸준함에 따른 것으로
분석되
능동형컴퓨팅이관   거대하고 복잡해지는 데이터의 효율적 가공과 관리루
위한 인간두뇌와 유사한
형태의 정보처리기술올 말히다 인공지능 상황인지컴유팅 등의 문목으로 구성팀
컴퓨팅 기소울 활용하 다양한 사업화기 활발히 진행되고 있어 역신성장 정책금용 공급 규모가 매년
약 10093 수준오로 증가하고 있으머; 새정부의 '미래 먹거리신업
신성장 전락추진" 예 따리 인공
지능 관련 기술로의 금움지원이 늘어날
선망몸
에너지 방산 우주임공 . 인공지능시) , 비이오
단소중립 대S 스미트농업은 차세대 6대 먹거리 산업으로 선정
것으로


문서를 대상으로 정확매칭 - 유사검색 - 생성형 답변
  - 메타데이터를 추가해서 문단 단위로 분할

In [ ]:
def load_and_prepare_document():
  loader = PyPDFLoader('/content/산림병해충 방제규정(산림청훈령)(제1664호)(20241210).pdf')
  documents = loader.load()  # 페이지 단위로 로드
  # 각 페이지에 메타데이터 추가
  for doc in documents:
    page_content = doc.page_content
    page_num = doc.metadata.get("page",0) + 1 # 페이지 번호 추출
    # first_line =  page_content.split('\n')[0].strip() # 첫 줄 추출
    # section_title = first_line if first_line.startswith("제") else f"페이지 {page_num}"
    section_title = page_content if page_content.startswith("제") else f"페이지 {page_num}"
    doc.metadata.update({"page":page_num, "section":section_title})

  # 문서 분할
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20,
                    separators=['.','\n',' ',''],keep_separator=True # 구분자를 청크 끝에 포함
                                                 )
  return text_splitter.split_documents(documents)
# 청크로 데이터베이스 생성
def create_and_search_vectorstore(texts):
  ''' chroma 벡터 db 생성, 질문에 대한 유사 문장 검색
  '''
  embedding = OpenAIEmbeddings()
  vectorstore = Chroma.from_documents(texts, embedding, persist_directory='./chroma_db')
  return vectorstore
# 유사 문장 검색 테스트 함수
def test_simularity_search(vectorstore,question = '병해충 예찰 및 발생조사 에 대해서?'):
  results = vectorstore.similarity_search(question,k=3)
  if results:
    for i, doc in enumerate(results,1):
      # print(f"{i} 페이지 {doc.metadata['page']}  {doc.metadata['section']} : {doc.page_content}")
      print(f'{doc.page_content}')
  else:
    print('유사문장 없음')

In [ ]:
texts = load_and_prepare_document()
vectorstore = create_and_search_vectorstore(texts)
question = '제 4조는 어떤 내용인가'
test_simularity_search(vectorstore,question)

법제처                                                            4                                                   국가법령정보센터
산림병해충 방제규정
제4조의2(위험평가의 대상) 위험평가의 대상은 다음 각 호와 같다.
1
법제처                                                            4                                                   국가법령정보센터
산림병해충 방제규정
제4조의2(위험평가의 대상) 위험평가의 대상은 다음 각 호와 같다.
1
법제처                                                            4                                                   국가법령정보센터
산림병해충 방제규정
제4조의2(위험평가의 대상) 위험평가의 대상은 다음 각 호와 같다.
1


In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# OpenAI API 키 설정


# 1. PDF 문서 로드 및 준비
def load_and_prepare_documents():
    """
    PDF 문서를 로드하고, 페이지별 메타데이터를 추가해 분할.
    Returns: 분할된 텍스트 청크 리스트
    """
    print("=== PDF 문서 로드 및 준비 ===")
    try:
        loader = PyPDFLoader("/content/산림병해충 방제규정(산림청훈령)(제1664호)(20241210).pdf")
        documents = loader.load()
        print(f"로드된 문서 페이지 수: {len(documents)}")
    except Exception as e:
        print(f"PDF 로드 에러: {e}")
        return []

    # 페이지별 메타데이터 추가 (페이지 번호, 조항명 추정)
    for doc in documents:
        page_content = doc.page_content
        page_num = doc.metadata.get("page", 0) + 1
        # 조항명 추정 (예: "제1조"로 시작)
        first_line = page_content.split("\n")[0].strip()
        section_title = first_line if first_line.startswith("제") else f"페이지 {page_num}"
        doc.metadata.update({"page": page_num, "section": section_title})

    # 문서 분할 (한글 문서에 최적화)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # 로그 기반 228 청크에 적합
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " ", ""],
        keep_separator=True
    )
    texts = text_splitter.split_documents(documents)
    print(f"분할된 청크 수: {len(texts)}")
    return texts

# 2. RAG 챗봇 생성 (CoT 프롬프트)
def create_rag_chatbot(texts):
    """
    PDF 문서를 기반으로 CoT 프롬프트 RAG 챗봇 생성.
    Args:
        texts: 분할된 문서 청크
    Returns: RetrievalQA 체인
    """
    print("\n=== RAG 챗봇 생성 (COT 프롬프트) ===")
    try:
        # 벡터 데이터베이스 생성
        embeddings = OpenAIEmbeddings()
        vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")
        print("벡터 데이터베이스 생성 완료")

        # LLM 초기화
        llm = ChatOpenAI(model_name="gpt-4.1-2025-04-14", temperature=0.7)

        # CoT 프롬프트
        template = """문서: {context}
질문: {question}
다음 단계를 따라 한글로 답변하세요:
1) 문서에서 질문과 관련된 문장을 검색합니다.
2) 관련 문장이 있으면 해당 항목들을 나열한다
3) 문장에 없으면 "문서에 정보 없음"을 명시하고, 일반 지식으로 간결히 답변합니다.
답변은 정확하게 먼저 문서에서 찾아서 관련항목들을 나열한다"""
        prompt_template = PromptTemplate(input_variables=["context", "question"], template=template)

        # RetrievalQA 체인
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt_template}
        )
        return qa_chain
    except Exception as e:
        print(f"RAG 챗봇 생성 에러: {e}")
        return None

# 3. 챗봇 테스트
def test_chatbot(qa_chain):
    """
    CoT 프롬프트로 챗봇 테스트.
    Args:
        qa_chain: RetrievalQA 체인
    """
    print("\n=== 챗봇 테스트 ===")
    questions = [
        "산림병해충 방제의 원칙은 무엇인가?",
        "위험평가의 대상은"
    ]
    for q in questions:
        print(f"\n질문: {q}")
        try:
            result = qa_chain.invoke({"query": q})
            print(f"COT 답변: {result['result']}")
            # print(f"참조 문서: {[f'페이지 {doc.metadata['page']} ({doc.metadata['section']}): {doc.page_content}' for doc in result['source_documents']]}")
        except Exception as e:
            print(f"COT 에러: {e}")
        print("-" * 50)

# 메인 실행
def main():
    # 문서 로드 및 분할
    texts = load_and_prepare_documents()
    if not texts:
        print("문서 처리 실패, 종료.")
        return

    # CoT 기반 챗봇 생성
    qa_chain = create_rag_chatbot(texts)
    if not qa_chain:
        print("챗봇 생성 실패, 종료.")
        return

    # 챗봇 테스트
    test_chatbot(qa_chain)

if __name__ == "__main__":
    main()

=== PDF 문서 로드 및 준비 ===
로드된 문서 페이지 수: 28
분할된 청크 수: 228

=== RAG 챗봇 생성 (COT 프롬프트) ===
벡터 데이터베이스 생성 완료

=== 챗봇 테스트 ===

질문: 산림병해충 방제의 원칙은 무엇인가?
COT 답변: 1) 문서에서 질문과 관련된 문장을 검색합니다.  
- "발생한 산림병해충에 효과적이며 적용가능한 약제를 선택하고, 「산림병해충 방제 농약등의 안전사용지침」을 준수할 것"
- "제11조(조사의 기본 원칙) ① 방제 대상목 조사는 전수조사를 원칙으로 한다. 다만, 다음 각 호의 경우에는 표준지(조사, 측정, 평가 등의 기준이 되는 지역)조사를 할 수 있다."

2) 관련 문장이 있으면 해당 항목들을 나열한다  
- 발생한 산림병해충에 효과적이며 적용가능한 약제를 선택할 것  
- 「산림병해충 방제 농약등의 안전사용지침」을 준수할 것  
- 방제 대상목 조사는 전수조사를 원칙으로 할 것 (단, 특정 경우 표준지 조사 가능)

3) 답변  
산림병해충 방제의 원칙은 다음과 같습니다.
- 발생한 산림병해충에 효과적이며 적용가능한 약제를 선택할 것  
- 「산림병해충 방제 농약등의 안전사용지침」을 준수할 것  
- 방제 대상목 조사는 전수조사를 원칙으로 할 것 (특정 경우에는 표준지 조사 가능)
--------------------------------------------------

질문: 위험평가의 대상은
COT 답변: 1) 문서에서 질문과 관련된 문장을 검색합니다.  
질문: 위험평가의 대상은

문서 내 관련 문장:  
"제4조의5(위험평가의 실시) ① 위험평가는 다음 각 호의 항목을 포함하여 실시하여야 한다.  
1. 대상 산림병해충의 외래병해충 여부  
2. 대상 산림병해충의 생리ㆍ생태적 특성  
3. 대상 산림병해충으로 인한 예상 피해 정도"

2) 관련 문장이 있으므로 해당 항목들을 나열합니다.

- 대상 산림병해충의 외래병해충 여부
- 대상 산림병해충의 생리ㆍ생태적 특성
- 대상 산림병해충으

RAG - QA 구축

In [ ]:
# pdf 파일 로드
loader = PyPDFLoader('/content/대한민국헌법(헌법)(제00010호)(19880225).pdf')
pages = loader.load_and_split()

In [ ]:
# 청크로 분할 1000 - size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs =  text_splitter.split_documents(pages)

In [ ]:
# 임베딩
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)
retriver = vectorstore.as_retriever()

In [ ]:
# 프롬프트 모델
llm = ChatOpenAI(model = 'gpt-4o-mini')

In [ ]:
# 프롬프트 엔지니어링
prompt = '''
당신은 질문 답변 어시스턴트 입니다.
제공된 문맥을 이용해서 질문에 답하세요
문맥에 답변이 없으면 "정보 없음"이라고 표시하고 일반 지식으로 간결히 답변하세요
답변은 최대 세 문장으로 작성하세요

문맥 : {context}
질문 : {question}
'''
prompt_template = PromptTemplate(input_variables=["context", "question"], template=prompt)

# RetrievalQA 체인
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)
result = qa_chain.invoke('대통령의 의무')
print(f"결과 : {result['result']}")
print("참조문서 : ")
for doc in result['source_documents']:
  print(doc.page_content)

결과 : 대통령의 의무에는 국민 앞에 성실히 직책을 수행할 것을 선서하는 것이 포함됩니다. 또한 대통령은 헌법과 법률에 따라 국군을 통수하고, 행정에 관한 명령을 발하고, 국회를 통보하는 등의 여러 규정을 준수해야 합니다. 이 외에도 대통령은 외교 및 국가 안전을 위한 중요 정책을 국민투표에 부치고, 국무총리와 국무위원을 임명 및 보좌하는 등의 의무가 있습니다.
참조문서 : 
노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다.”
 
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의
안녕질서를 유지하기 위하여 긴급한 조치가 필요하고 국회의 집회를 기다릴 여유가 없을 때에 한하여 최소한으로
필요한 재정ㆍ경제상의 처분을 하거나 이에 관하여 법률의 효력을 가지는 명령을 발할 수 있다.
제80조 대통령은 법률이 정하는 바에 의하여 훈장 기타의 영전을 수여한다.
 
제81조 대통령은 국회에 출석하여 발언하거나 서한으로 의견을 표시할 수 있다.
 
제82조 대통령의 국법상 행위는 문서로써 하며, 이 문서에는 국무총리와 관계 국무위원이 부서한다. 군사에 관한 것도
또한 같다.
 
제83조 대통령은 국무총리ㆍ국무

RAG-memory
  - 페이지별 메타에이터 활용
  - 대화 메모리 포함 세션별 기록 저장

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
# 위의 코드를 재 사용하면서
# 청크로 분할 1000 - size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs =  text_splitter.split_documents(pages)
# 임베딩
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)
retriver = vectorstore.as_retriever()

# 채팅 히스토리와 사용지 질문 문맥을 고려해서 질문 재 구성
context_prompt = ''' 대화 기록과 최신 사용자 질문을 바탕으로, \
대화 기록 없이도 이해할 수 있는 독립적인 질문을 구성하세요. \
질문에 답변하지 말고, 필요한 경우 질문만 재구성하거나 그대로 반환하세요
'''
contextualize_prompt =  ChatPromptTemplate.from_messages([
 ("system", context_prompt)   ,
 MessagesPlaceholder(variable_name="chat_history"),
 ("human", "{input}")
])
# 대화 기록을 고려한 검색기 생성
# llm = ChatOpenAI(model = 'gpt-4o-mini')
llm = ChatOpenAI()
history_aware_retriever =  create_history_aware_retriever(llm, retriver,contextualize_prompt)
# 대화 기록 example
chat_history = [
    HumanMessage(content="대통령의 임기는 몇년이야?"),
    AIMessage(content="대통령의 임기는 5년 입니다.")
]
result = contextualize_prompt.invoke({"input" : "국회의원의 임기는??", "chat_history":chat_history})

In [ ]:
result = history_aware_retriever.invoke({"input" : "국회의원의 임기는??", "chat_history":chat_history})
print(result[0].page_content)

In [ ]:
# RAG : 문서기반 검색시스템
  # 벡터DB
# LangChain : RAG데이터를 LLM으로 전달해서 생성형 ai 실행
  # 프롬프트 엔지니어링

# 다양한 형태의 문서를 load
# json, docx, pdf, csv , txt, ocr(image 텍스트 추출), notion
# 문서기반으로 요약및 QA 등등....
# 해당 내용이 없으면 일반 gpt로 추론( 일반모델이 아니라 특화된 파인튜닝 모델도 가능)

# 나만의 RAG 기반 랭체인 VS 일반 gpt